In [ ]:
import tensorflow as tf 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding,LSTM,TimeDistributed,Dense,Flatten,Dropout,RepeatVector,GRU,Bidirectional,Permute
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import kerastuner as kt
from kerastuner.tuners import BayesianOptimization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input
from keras_preprocessing import sequence
from tensorflow.python.keras.layers import concatenate,Activation, dot

import os

filename='pre_data_dl.xlsx'
data_dl=pd.read_excel(filename)
pd.set_option('display.max_columns', None)
print(data_dl.head())

filename1='pre_data_dl_aug1.xlsx'
filename2='pre_data_dl_aug2.xlsx'

data_dl_aug1=pd.read_excel(filename1)
pd.set_option('display.max_columns', None)
print(data_dl_aug1.head())

data_dl_aug2=pd.read_excel(filename2)
print(data_dl_aug2.head())

X = (data_dl["Combined Description Cleaned"])
y= (data_dl['Assignment group'])

#categorical encoding y
y=pd.get_dummies(data_dl['Assignment group'])
  

#parameters
max_features=10000
emb_dim=300
batch_size=1024
epochs=10
out_dim=74

In [ ]:
#function for tokenizer
def dfTokenizer(df):
 tokenizer=Tokenizer(num_words=max_features,char_level=False)
 tokenizer.fit_on_texts(df)
 sequences=tokenizer.texts_to_sequences(df)
 return sequences,tokenizer
#tokenization
X,tokenizer = dfTokenizer(data_dl["Combined Description Cleaned"]) 
vocab_size=len(tokenizer.word_index)
print("vocabulary size is: ",vocab_size)

#function for padding
def pad(x, length=None):
 if length is None:
   length=max([len(sentence)  for sentence in x])
 return pad_sequences(x,maxlen=length,padding='post')
 
 #padding 
X=pad_sequences(X,padding='post')


#function for splitting the data
def split(X,y):
 X_train_spl,X_test_spl,y_train_spl,y_test_spl=train_test_split(X,y,test_size=0.2,random_state=123)
 return X_train_spl,X_test_spl,y_train_spl,y_test_spl
#split the data
X_train,X_test,y_train,y_test=split(X,y)


#configuring the callback
early_stopping =  EarlyStopping(
    monitor='val_accuracy', 
    patience=3, 
    min_delta=0.01, 
    mode='max'
)


#function for plotting accuracy,loss
def plot(model,history):
 acc = history.history['accuracy']
 val_acc = history.history['val_accuracy']
 loss = history.history['loss']
 val_loss = history.history['val_loss']
 epochs = range(1, len(acc) + 1)
 plt.plot(epochs, acc, 'bo', label='Training acc')
 plt.plot(epochs, val_acc, 'b', label='Validation acc')
 plt.title( ' Training and validation accuracy')
 plt.legend()
 plt.figure()
 plt.plot(epochs, loss, 'bo', label='Training loss')
 plt.plot(epochs, val_loss, 'b', label='Validation loss')
 plt.title('Training and validation loss')
 plt.legend()
 plt.show()
 return 

In [ ]:
inp_len=X.shape[1]
inputs = Input(shape=(inp_len,))
#attention with encoder only
sequence_input = Input(shape=(inp_len), dtype='int32')        
embedded_sequences =Embedding(max_features,emb_dim,input_length=inp_len)(sequence_input)     
BILSTM= Bidirectional(LSTM(128, return_sequences=True,dropout=0.2,name="bi_lstm_0",return_state=True))(embedded_sequences)
#rp=RepeatVector(max_features)(lstm)

#xy = tf.reshape(lstm, [189, 256])
#rp=RepeatVector(max_features)(xy)

BILSTM,forward_h, forward_c, backward_h, backward_c= Bidirectional \
                                                    (LSTM
                                                    (128,
                                                     dropout=0.2,
                                                     return_state=True,
                                                    
                                                     return_sequences=True))(BILSTM)
                                                   
                                                   
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

context_vector,attention_weights = Attention(32)(BILSTM,state_h)
fl=Flatten()(context_vector)
output = Dense(74, activation='sigmoid')(fl)

model_att =tf.keras.Model(inputs=sequence_input, outputs=output)

# summarize layers
print(model_att.summary())


#Attention with encoder decoder
inp = Embedding(max_features,emb_dim,input_length=inp_len)(inputs)
lstm_out = LSTM(64, return_sequences=True)(inp)

attention = Dense(1, activation='relu')(lstm_out)
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(64)(attention)
#attention = LSTM(64,return_sequences=False)(attention)
attention = Permute([2,1])(attention)

combined = concatenate([lstm_out, attention])
combined_mul = Flatten()(combined)
decode = RepeatVector(64)(combined_mul)

decode = LSTM(64, return_sequences=True)(decode)
decode=Flatten()(decode)
decode = (Dense(74))(decode)

decode = Activation('softmax')(decode)

model_enc_dec_att = tf.keras.Model(inputs=inputs, outputs=decode)

#model= tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[output])
model_enc_dec_att.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])
print(model_enc_dec_att.summary())

In [ ]:
def ENC_ATT_model(hp): #attention with encoder
 model = model_att
 print(model.summary())
 lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])   
 model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
 
 return model

def ENC_DEC_ATT_model(hp):#attention with encoder decoder
 model = model_enc_dec_att
 print(model.summary())
 lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])   
 model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
 
 return model

In [ ]:
class MyTuner(kt.tuners.BayesianOptimization):
  def run_trial(self, trial, *args, **kwargs):
     kwargs['batch_size'] = trial.hyperparameters.Int('batch_size', 256, 512,step=256 )
     kwargs['epochs'] = trial.hyperparameters.Int('epochs', 5, 10,20)
     super(MyTuner, self).run_trial(trial, *args, **kwargs)

In [ ]:
tuner =MyTuner(
    ENC_ATT_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    seed=123,
    directory="C:/Hyperparameter/4F"
)

tuner.search(X_train,y_train,validation_data=(X_test,y_test))
print(tuner.results_summary(5))
best_model_ENC_ATT = tuner.get_best_models()[0]
print("Hyperparameters for ENC_ATT are",best_model_ENC_ATT)


modelname="ENC_ATTModel"
vals=tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print(vals)
batch_size_hyp=list(vals.values())[1]
epochs_hyp=list(vals.values())[2]
history=best_model_ENC_ATT.fit(X_train,y_train,batch_size=batch_size_hyp,epochs=epochs_hyp,validation_data=(X_test,y_test),callbacks=[early_stopping])
plot(best_model_ENC_ATT,history)

scores_ENC_ATT =best_model_ENC_ATT.evaluate(X_test, y_test, verbose=0)
scores_ENC_ATT_val = best_model_ENC_ATT.evaluate(X_train, y_train, verbose=0)
print("Accuracy of ENC_ATT for unaugmented data is :", (scores_ENC_ATT[1]*100))
print("Validation Accuracy of ENC_ATT for unagumented data is:", (scores_ENC_ATT_val[1]*100))
#print("Validation Accuracy of ENC_ATT for unagumented data is:" ,np.array(history.history['val_acc']).mean()*100)
print( "Mean loss of  Validation data of ENC_ATT for unaugmented data is :",np.array(history.history['val_loss']).mean())
print( "Mean loss of  Training data of ENC_ATT for unaugmented data is :",np.array(history.history['loss']).mean())
      

In [ ]:
tuner =MyTuner(
    ENC_DEC_ATT_model,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    seed=123,
    directory="C:/Hyperparameter/4G"
)

tuner.search(X_train,y_train,validation_data=(X_test,y_test))
print(tuner.results_summary(5))
best_model_ENC_DEC_ATT = tuner.get_best_models()[0]
print("Hyperparameters for ENC_DEC_ATT are",best_model_ENC_DEC_ATT)


modelname="ENC_DEC_ATTModel"
vals=tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print(vals)
batch_size_hyp=list(vals.values())[1]
epochs_hyp=list(vals.values())[2]
history=best_model_ENC_DEC_ATT.fit(X_train,y_train,batch_size=batch_size_hyp,epochs=epochs_hyp,validation_data=(X_test,y_test),callbacks=[early_stopping])
plot(best_model_ENC_DEC_ATT,history)

scores_ENC_DEC_ATT =best_model_ENC_DEC_ATT.evaluate(X_test, y_test, verbose=0)
scores_ENC_DEC_ATT_val = best_model_ENC_DEC_ATT.evaluate(X_train, y_train, verbose=0)
print("Accuracy of ENC_DEC_ATT for unaugmented data is :", (scores_ENC_DEC_ATT[1]*100))
print("Validation Accuracy of ENC_DEC_ATT for unagumented data is:", (scores_ENC_DEC_ATT_val[1]*100))
#print("Validation Accuracy of ENC_DEC_ATT for unagumented data is:" ,np.array(history.history['val_acc']).mean()*100)
print( "Mean loss of  Validation data of ENC_DEC_ATT for unaugmented data is :",np.array(history.history['val_loss']).mean())
print( "Mean loss of  Training data of ENC_DEC_ATT for unaugmented data is :",np.array(history.history['loss']).mean())    

In [ ]:
X_aug1 = (data_dl_aug1["Combined Description Cleaned"])
y_aug1= (data_dl_aug1['Assignment group'])

#categorical encoding y
y_aug1=pd.get_dummies(data_dl_aug2['Assignment group'])


X_aug2 = (data_dl_aug2["Combined Description Cleaned"])
y_aug2= (data_dl_aug2['Assignment group'])

#categorical encoding y
y_aug2=pd.get_dummies(data_dl_aug2['Assignment group'])

#tokenization
X_aug1,tokenizer = dfTokenizer(data_dl_aug1["Combined Description Cleaned"]) 
vocab_size_aug1=len(tokenizer.word_index)
print("vocabulary size is: ",vocab_size_aug1)

X_aug2,tokenizer = dfTokenizer(data_dl_aug2["Combined Description Cleaned"]) 
vocab_size_aug2=len(tokenizer.word_index)
print("vocabulary size is: ",vocab_size_aug2)

#padding
X_aug1=pad_sequences(X_aug1,padding='post')
X_aug2=pad_sequences(X_aug2,padding='post')
y_aug1=y_aug1[0:17586]

def split_stratify(X,y):
 X_train_spl,X_test_spl,y_train_spl,y_test_spl=train_test_split(X,y,test_size=0.2,stratify=y,random_state=123)
 return X_train_spl,X_test_spl,y_train_spl,y_test_spl

X_train_aug1,X_test_aug1,y_train_aug1,y_test_aug1=split_stratify(X_aug1,y_aug1)
X_train_aug2,X_test_aug2,y_train_aug2,y_test_aug2=split_stratify(X_aug2,y_aug2)


inp_len1=X_aug1.shape[1]
print(inp_len1)

In [ ]:
sequence_input = Input(shape=(inp_len1), dtype='int32')        
embedded_sequences =Embedding(max_features,emb_dim,input_length=inp_len1)(sequence_input)     
BILSTM= Bidirectional(LSTM(128, return_sequences=True,dropout=0.2,name="bi_lstm_0",return_state=True))(embedded_sequences)
#rp=RepeatVector(max_features)(lstm)

#xy = tf.reshape(lstm, [189, 256])
#rp=RepeatVector(max_features)(xy)

BILSTM,forward_h, forward_c, backward_h, backward_c= Bidirectional \
                                                    (LSTM
                                                    (128,
                                                     dropout=0.2,
                                                     return_state=True,
                                                    
                                                     return_sequences=True))(BILSTM)
                                                   
                                                   
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

context_vector,attention_weights = Attention(32)(BILSTM,state_h)
fl=Flatten()(context_vector)
output = Dense(74, activation='sigmoid')(fl)

model_att1 =tf.keras.Model(inputs=sequence_input, outputs=output)

# summarize layers
print(model_att1.summary())


#Attention with encoder decoder
inp = Embedding(max_features,emb_dim,input_length=inp_len11)(inputs)
lstm_out = LSTM(64, return_sequences=True)(inp)

attention = Dense(1, activation='relu')(lstm_out)
attention = Flatten()(attention)
attention = Activation('softmax')(attention)
attention = RepeatVector(64)(attention)
#attention = LSTM(64,return_sequences=False)(attention)
attention = Permute([2,1])(attention)

combined = concatenate([lstm_out, attention])
combined_mul = Flatten()(combined)
decode = RepeatVector(64)(combined_mul)

decode = LSTM(64, return_sequences=True)(decode)
decode=Flatten()(decode)
decode = (Dense(74))(decode)

decode = Activation('softmax')(decode)

model_enc_dec_att1 = tf.keras.Model(inputs=inputs, outputs=decode)

#model= tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[output])
model_enc_dec_att1.compile(optimizer='adam', loss='categorical_crossentropy',metrics = ['accuracy'])
print(model_enc_dec_att1.summary())

In [ ]:
tuner =MyTuner(
    ENC_ATT_model1,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    seed=123,
    directory="C:/Hyperparameter/9A"
)

tuner.search(X_train_aug1,y_train_aug1,validation_data=(X_test_aug1,y_test_aug1))
print(tuner.results_summary(5))
best_model_ENC_ATT = tuner.get_best_models()[0]
print("Hyperparameters for ENC_ATT are",best_model_ENC_ATT)


modelname="ENC_ATTModel_aug1"
vals=tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print(vals)
batch_size_hyp=list(vals.values())[1]
epochs_hyp=list(vals.values())[2]
history=best_model_ENC_ATT.fit(X_train_aug1,y_train_aug1,batch_size=batch_size_hyp,epochs=epochs_hyp,validation_data=(X_test_aug1,y_test_aug1),callbacks=[early_stopping])
plot(best_model_ENC_ATT,history)

scores_ENC_ATT_aug1 =best_model_ENC_ATT.evaluate(X_test_aug1, y_test_aug1, verbose=0)
scores_ENC_ATT_val_aug1 = best_model_ENC_ATT.evaluate(X_train_aug1, y_train_aug1, verbose=0)
print("Accuracy of ENC_ATT for level1 augmented data is :", (scores_ENC_ATT_aug1[1]*100))
print("Validation Accuracy of ENC_ATT for level1 augmented data is:", (scores_ENC_ATT_val_aug1[1]*100))
#print("Validation Accuracy of ENC_ATT for level1 augmented data is:" ,np.array(history.history['val_acc']).mean()*100)
print( "Mean loss of  Validation data of ENC_ATT for level1 augmented data is :",np.array(history.history['val_loss']).mean())
print( "Mean loss of  training data of ENC_ATT for level1 augmented data is :",np.array(history.history['loss']).mean())

In [ ]:
tuner =MyTuner(
    ENC_ATT_model1,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    seed=123,
    directory="C:/Hyperparameter/9F"
)

tuner.search(X_train_aug2,y_train_aug2,validation_data=(X_test_aug2,y_test_aug2))
print(tuner.results_summary(5))
best_model_ENC_ATT = tuner.get_best_models()[0]
print("Hyperparameters for ENC_ATT are",best_model_ENC_ATT)


modelname="ENC_ATTModel_aug2"
vals=tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print(vals)
batch_size_hyp=list(vals.values())[1]
epochs_hyp=list(vals.values())[2]
history=best_model_ENC_ATT.fit(X_train_aug2,y_train_aug2,batch_size=batch_size_hyp,epochs=epochs_hyp,validation_data=(X_test_aug2,y_test_aug2),callbacks=[early_stopping])
plot(best_model_ENC_ATT,history)

scores_ENC_ATT_aug2 =best_model_ENC_ATT.evaluate(X_test_aug2, y_test_aug2, verbose=0)
scores_ENC_ATT_val_aug2 = best_model_ENC_ATT.evaluate(X_train_aug2, y_train_aug2, verbose=0)
print("Accuracy of ENC_ATT for level2 augmented data is :", (scores_ENC_ATT_aug2[1]*100))
print("Validation Accuracy of ENC_ATT for level2 augmented data is:", (scores_ENC_ATT_val_aug2[1]*100))
#print("Validation Accuracy of ENC_ATT for level1 augmented data is:" ,np.array(history.history['val_acc']).mean()*100)
print( "Mean loss of  Validation data of ENC_ATT for level2 augmented data is :",np.array(history.history['val_loss']).mean())
print( "Mean loss of  training data of ENC_ATT for level2 augmented data is :",np.array(history.history['loss']).mean())

In [ ]:
tuner =MyTuner(
    ENC_DEC_ATT_model1,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    seed=123,
    directory="C:/Hyperparameter/9K"
)

tuner.search(X_train_aug1,y_train_aug1,validation_data=(X_test_aug1,y_test_aug1))
print(tuner.results_summary(5))
best_model_ENC_DEC_ATT = tuner.get_best_models()[0]
print("Hyperparameters for ENC_DEC_ATT are",best_model_ENC_DEC_ATT)


modelname="ENC_DEC_ATTModel_aug1"
vals=tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print(vals)
batch_size_hyp=list(vals.values())[1]
epochs_hyp=list(vals.values())[2]
history=best_model_ENC_DEC_ATT.fit(X_train_aug1,y_train_aug1,batch_size=batch_size_hyp,epochs=epochs_hyp,validation_data=(X_test_aug1,y_test_aug1),callbacks=[early_stopping])
plot(best_model_ENC_DEC_ATT,history)

scores_ENC_DEC_ATT_aug1 =best_model_ENC_DEC_ATT.evaluate(X_test_aug1, y_test_aug1, verbose=0)
scores_ENC_DEC_ATT_val_aug1 = best_model_ENC_DEC_ATT.evaluate(X_train_aug1, y_train_aug1, verbose=0)
print("Accuracy of ENC_DEC_ATT for level1 augmented data is :", (scores_ENC_DEC_ATT_aug1[1]*100))
print("Validation Accuracy of ENC_DEC_ATT for level1 augmented data is:", (scores_ENC_DEC_ATT_val_aug1[1]*100))
#print("Validation Accuracy of ENC_DEC_ATT for level1 augmented data is:" ,np.array(history.history['val_acc']).mean()*100)
print( "Mean loss of  Validation data of ENC_DEC_ATT for level1 augmented data is :",np.array(history.history['val_loss']).mean())
print( "Mean loss of  training data of ENC_DEC_ATT for level1 augmented data is :",np.array(history.history['loss']).mean())

In [ ]:
tuner =MyTuner(
    ENC_DEC_ATT_model1,
    objective='val_accuracy',
    max_trials=10,
    executions_per_trial=1,
    seed=123,
    directory="C:/Hyperparameter/9H"
)

tuner.search(X_train_aug2,y_train_aug2,validation_data=(X_test_aug2,y_test_aug2))
print(tuner.results_summary(5))
best_model_ENC_DEC_ATT = tuner.get_best_models()[0]
print("Hyperparameters for ENC_DEC_ATT are",best_model_ENC_DEC_ATT)


modelname="ENC_DEC_ATTModel_aug2"
vals=tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
print(vals)
batch_size_hyp=list(vals.values())[1]
epochs_hyp=list(vals.values())[2]
history=best_model_ENC_DEC_ATT.fit(X_train_aug2,y_train_aug2,batch_size=batch_size_hyp,epochs=epochs_hyp,validation_data=(X_test_aug2,y_test_aug2),callbacks=[early_stopping])
plot(best_model_ENC_DEC_ATT,history)

scores_ENC_DEC_ATT_aug2 =best_model_ENC_DEC_ATT.evaluate(X_test_aug2, y_test_aug2, verbose=0)
scores_ENC_DEC_ATT_val_aug2 = best_model_ENC_DEC_ATT.evaluate(X_train_aug2, y_train_aug2, verbose=0)
print("Accuracy of ENC_DEC_ATT for level2 augmented data is :", (scores_ENC_DEC_ATT_aug2[1]*100))
print("Validation Accuracy of ENC_DEC_ATT for level2 augmented data is:", (scores_ENC_DEC_ATT_val_aug2[1]*100))
#print("Validation Accuracy of ENC_DEC_ATT for level1 augmented data is:" ,np.array(history.history['val_acc']).mean()*100)
print( "Mean loss of  Validation data of ENC_DEC_ATT for level2 augmented data is :",np.array(history.history['val_loss']).mean())
print( "Mean loss of  training data of ENC_DEC_ATT for level2 augmented data is :",np.array(history.history['loss']).mean())

In [ ]:
print("Accuracy of Attention with encoder for unaugmented data is :", (scores_ENC_ATT[1]*100))
print("Accuracy of Attention with encoder & decoder for unagumented data is:", (scores_ENC_DEC_ATT[1]*100))
print("Accuracy of Attention with encoder for level1 augmented data is :", (scores_ENC_ATT_aug1[1]*100))
print("Accuracy of Attention with encoder for level2 augmented data is :", (scores_ENC_ATT_aug2[1]*100))
print("Accuracy of Attention with encoder & decoder for level1 augmented data is :", (scores_ENC_DEC_ATT_aug1[1]*100))
print("Accuracy of Attention with encoder & decoder for level2 augmented data is :", (scores_ENC_DEC_ATT_aug2[1]*100))